In [146]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
import ipywidgets
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import model_selection
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm

/home/sandhya/Project/final/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [49]:
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension

In [383]:
%run ../paths.py
CONST = CONST()

In [282]:
train_data = pd.read_csv(CONST.TRAIN_DATA)
test_data = pd.read_csv(CONST.TEST_DATA)

In [283]:
train_data.drop(train_data.columns[0],axis=1,inplace=True)
test_data.drop(test_data.columns[0],axis=1,inplace=True)


In [284]:
train_data.head()

,tweet,class
0,Mannnnnn these hoes aint loyal,1
1,have a fabulous weekend beautiful souls #nofil...,0
2,#fathersday #dad @user,0
3,RT @SheswantstheD: these hoes ain't loyal http...,1
4,newhairâ¨ð #lapis#rollen#new#hair#brond#gr...,0


In [285]:
test_data.head()

,tweet,class
0,What I look like ho? I look like YES and you l...,1
1,@user lots of #resistance and #challenges fro...,0
2,what's wrong with ppl nowadays? i was conducti...,0
3,@Montulos Lakers are trash,1
4,will be playing rf simpson in #singinintherain...,0


In [286]:
vec = TfidfVectorizer(ngram_range=(1,2),max_features=10000)
train_tfidf = vec.fit_transform(train_data.tweet)
test_tfidf = vec.transform(test_data.tweet)


In [287]:
train_tfidf.shape

(42364, 10000)

In [288]:
clf = naive_bayes.MultinomialNB()
clf.fit(train_tfidf,train_data["class"])
predictions = clf.predict(test_tfidf)

In [289]:
metrics.f1_score(test_data['class'],predictions)

0.8971119133574007

In [290]:
%run ../src/processing/html_tag_remover.py

In [291]:
train_data["tweet"] = train_data["tweet"].progress_apply(lambda x: strip_tags(x))
test_data["tweet"] = test_data["tweet"].progress_apply(lambda x: strip_tags(x))

In [292]:
vec = TfidfVectorizer(ngram_range=(1,2),max_features=10000)
train_tfidf = vec.fit_transform(train_data.tweet)
test_tfidf = vec.transform(test_data.tweet)
clf.fit(train_tfidf,train_data["class"])
predictions = clf.predict(test_tfidf)
metrics.f1_score(test_data['class'],predictions)

0.896964856230032

In [293]:
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

In [294]:
%run ../src/processing/contractions.py

talk to you later wait


In [295]:
train_data['tweet'] = train_data['tweet'].progress_apply(lambda x: replace_slang(x))
test_data['tweet'] = test_data['tweet'].progress_apply(lambda x: replace_slang(x))


In [296]:
train_data['tweet'] = train_data['tweet'].progress_apply(lambda x: clean_contractions(x))
test_data['tweet'] = test_data['tweet'].progress_apply(lambda x: clean_contractions(x))


In [297]:
vec = TfidfVectorizer(ngram_range=(1,2),max_features=10000)
train_tfidf = vec.fit_transform(train_data.tweet)
test_tfidf = vec.transform(test_data.tweet)
clf.fit(train_tfidf,train_data["class"])
predictions = clf.predict(test_tfidf)
metrics.f1_score(test_data['class'],predictions)

0.8927328760314146

In [298]:
%run ../src/processing/glove_utils.py

In [299]:
word,frequency = get_top_n_words(25,vec,train_tfidf)

In [300]:
#word

In [301]:
%run ../src/processing/regular_expressions.py

 <hashtags>: what do you… … call a pig with three eyes???piiig!!!allcaps <allcaps>  .... full time time


In [302]:
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

In [303]:
train_data['tweet'] = train_data['tweet'].progress_apply(lambda x: tweet_preprocessing(x))
test_data['tweet'] = test_data['tweet'].progress_apply(lambda x: tweet_preprocessing(x))


# some features

In [304]:
train_data["hashtags"] = train_data["tweet"].progress_apply(lambda comment: comment.count('<hashtags>'))
train_data["user"] = train_data["tweet"].progress_apply(lambda comment: comment.count("<user>"))
train_data["allcaps"]  = train_data["tweet"].progress_apply(lambda comment: comment.count("<allcaps>"))


In [305]:
test_data["hashtags"] = test_data["tweet"].progress_apply(lambda comment: comment.count('<hashtags>'))
test_data["user"] = test_data["tweet"].progress_apply(lambda comment: comment.count("<user>"))
test_data["allcaps"]  = test_data["tweet"].progress_apply(lambda comment: comment.count("<allcaps>"))



In [306]:
train_data["tweet"] = train_data["tweet"].progress_apply(lambda comment: re.sub(r"(<hashtags>|<user>|<allcaps>)"," ", comment))
test_data["tweet"] = test_data["tweet"].progress_apply(lambda comment: re.sub(r"(<hashtags>|<user>|<allcaps>)"," ", comment))

In [307]:
train_data["tweet"] = train_data["tweet"].progress_apply(lambda comment: re.sub(r"\s+"," ", comment))
test_data["tweet"] = test_data["tweet"].progress_apply(lambda comment: re.sub(r"\s+"," ", comment))


In [308]:
vec = TfidfVectorizer(ngram_range=(1,2),max_features=10000)
train_tfidf = vec.fit_transform(train_data.tweet)
test_tfidf = vec.transform(test_data.tweet)
clf.fit(train_tfidf,train_data["class"])
predictions = clf.predict(test_tfidf)
metrics.f1_score(test_data['class'],predictions)

0.881002931958346

In [80]:
word,frequency = get_top_n_words(25,vec,train_tfidf)

In [309]:
stopwords = ['the','to','and','is','my','just','us','its','it','up','in','for','of','not','at','this','on','am','that','be','with','are','me','it','all','like','so','do','rt','ff']

In [310]:
train_data['tweet'] = train_data['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
test_data['tweet'] = test_data['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))


In [311]:
#?word

In [342]:
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

In [340]:
drop_rows = train_data[train_data['tweet']==""]
train_data.drop(drop_rows.index, axis=0,inplace=True)

In [341]:
drop_rows = test_data[test_data['tweet']==""]
test_data.drop(drop_rows.index, axis=0,inplace=True)

In [314]:
%run ../src/processing/spellings.py

In [315]:
train_data['tweet'] = train_data['tweet'].progress_apply(lambda x: replace_elongated_text(x))
test_data['tweet'] = test_data['tweet'].progress_apply(lambda x: replace_elongated_text(x))



In [317]:
vec = TfidfVectorizer(ngram_range=(1,2),max_features=10000)
train_tfidf = vec.fit_transform(train_data.tweet)
test_tfidf = vec.transform(test_data.tweet)
clf.fit(train_tfidf,train_data["class"])
predictions = clf.predict(test_tfidf)
metrics.f1_score(test_data['class'],predictions)

0.8874158678360188

In [106]:
word,frequency = get_top_n_words(25,vec,train_tfidf)

In [108]:
#word

In [258]:
%run ../src/processing/emojis.py

In [318]:
train_data['tweet'] = train_data['tweet'].progress_apply(lambda x: replace_emojis_with_text(x))
test_data['tweet'] = test_data['tweet'].progress_apply(lambda x: replace_emojis_with_text(x))



In [319]:
vec = TfidfVectorizer(ngram_range=(1,2),max_features=10000)
train_tfidf = vec.fit_transform(train_data.tweet)
test_tfidf = vec.transform(test_data.tweet)
clf.fit(train_tfidf,train_data["class"])
predictions = clf.predict(test_tfidf)
metrics.f1_score(test_data['class'],predictions)

0.8860267811509762

In [320]:
train_data['num_exclamation_marks'] = train_data['tweet'].progress_apply(lambda comment: comment.count('!'))
train_data['num_question_marks'] = train_data['tweet'].progress_apply(lambda comment: comment.count('?'))
train_data['num_punctuation'] = train_data['tweet'].progress_apply(lambda comment: sum(comment.count(w) for w in '.,;:'))
train_data['num_symbols'] = train_data['tweet'].progress_apply(lambda comment: sum(comment.count(w) for w in '*&$%'))

In [321]:
test_data['num_exclamation_marks'] = test_data['tweet'].progress_apply(lambda comment: comment.count('!'))
test_data['num_question_marks'] = test_data['tweet'].progress_apply(lambda comment: comment.count('?'))
test_data['num_punctuation'] = test_data['tweet'].progress_apply(lambda comment: sum(comment.count(w) for w in '.,;:'))
test_data['num_symbols'] = test_data['tweet'].progress_apply(lambda comment: sum(comment.count(w) for w in '*&$%'))

In [267]:
%run ../src/processing/special_char.py

dfvfe ffve rfvfe sdv eee


In [322]:
train_data['tweet'] = train_data['tweet'].progress_apply(lambda x: clean_special_chars(x))
test_data['tweet'] = test_data['tweet'].progress_apply(lambda x: clean_special_chars(x))



In [343]:
train_data['total_length'] = train_data['tweet'].progress_apply(len)
train_data['capitals'] = train_data['tweet'].progress_apply(lambda comment: sum(1 for c in comment if c.isupper()))
train_data['caps_vs_length'] =  train_data['capitals']/train_data['total_length']
train_data['num_words'] = train_data['tweet'].progress_apply(lambda comment: len(comment.split()))
train_data['num_unique_words'] = train_data['tweet'].progress_apply(lambda comment: len(set(w for w in comment.split())))
train_data['words_vs_unique'] = train_data['num_unique_words'] / train_data['num_words']

In [344]:
test_data['total_length'] = test_data['tweet'].progress_apply(len)
test_data['capitals'] = test_data['tweet'].progress_apply(lambda comment: sum(1 for c in comment if c.isupper()))
test_data['caps_vs_length'] =  test_data['capitals']/test_data['total_length']
test_data['num_words'] = test_data['tweet'].progress_apply(lambda comment: len(comment.split()))
test_data['num_unique_words'] = test_data['tweet'].progress_apply(lambda comment: len(set(w for w in comment.split())))
test_data['words_vs_unique'] = test_data['num_unique_words'] / train_data['num_words']



In [380]:
for c in train_data.columns:
        print("\n",c,"\n")
        print(train_data[c].value_counts())
        print("\n \n")



 tweet 

bitch                                                                                          25
pussy                                                                                          25
update                                                                                         24
fathers day                                                                                    17
always                                                                                         16
                                                                                               ..
last walk beach before heading home tomorrow  sanibel                                           1
damn you directtv blackout yankee game a freaking walk off  !  you killing satellite douche     1
only school wouldnt give kids their schedules                                                   1
i herd most random and creepiest sound da fuck going outside windows i looking out bitch        1
hate when 

In [367]:
vec = TfidfVectorizer(ngram_range=(1,2),max_features=10000)
train_tfidf = vec.fit_transform(train_data.tweet)
test_tfidf = vec.transform(test_data.tweet)
clf.fit(train_tfidf,train_data["class"])
predictions = clf.predict(test_tfidf)
metrics.f1_score(test_data['class'],predictions)

0.8850102669404518

In [360]:
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
#stemming decreased f1 score

In [373]:
train_data.to_csv(Path('../dataset/ML_CLEANED_train_data'))
test_data.to_csv(Path('../dataset/ML_CLEANED_test_data'))


In [382]:
pd.read_csv(CONST.CLEANED_TRAIN).shape

FileNotFoundError: [Errno 2] File /home/sandhya/Project/../dataset/ML_CLEANED_train_data does not exist: '/home/sandhya/Project/../dataset/ML_CLEANED_train_data'